In [1]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='DWbxNmQaykrJlauK687KocVhf5ObIZtTmPsR5UjEax45',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'smartlender-donotdelete-pr-ba42zh3cvkqfh5'
object_key = 'train.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score

#the model used to fit&predict
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#pipeline with its' preprocessor's transformers
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer

#used for estimating model accuracy and getting reports
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [3]:
[print(col, df[col].nunique()) for col in df.columns if df[col].dtype=='object']
df.drop('Loan_ID', axis=1, inplace=True)

Loan_ID 614
Gender 2
Married 2
Dependents 4
Education 2
Self_Employed 2
Property_Area 3
Loan_Status 2


In [4]:
y = LabelEncoder().fit_transform(df['Loan_Status'])
X = df.drop(['Loan_Status'], axis=1)

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0)

In [5]:
print(y)

[1 0 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 0 1 0 1 0 0 0 1 1 1 0 1 0 0 0 1 0 1 0
 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 1 1 0 0 0 0 0 1 1 0 1 1 1 0
 1 0 0 0 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 0 0 1 1 1 1 1 1 1
 0 1 0 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 1 0 1 1 1 0 1 0 1 1 0 1 0 0 0 1 0 1
 1 0 1 1 1 1 0 0 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 0 1 1 1 1 0 1 0 1 0 1
 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 1 0 0 0
 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 1 0 1 1
 1 1 0 0 0 1 1 1 1 0 1 0 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1
 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0 1 1 1 1 0 0 0 1 0
 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 0 1 1
 1 0 0 0 1 0 1 0 1 0 0 1 1 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 0 0 0 0 1 0 1 1 1 1 0 1 0 1 1 1 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 0 1
 1 1 1 1 1 0 0 1 0 1 1 1 

In [6]:
num_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]
cat_cols = [col for col in X.columns if X[col].dtype == 'object']
print(num_cols)
print(cat_cols)

['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']
['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']


In [7]:
num_transformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

cat_transformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_cols),
    ('cat', cat_transformer, cat_cols)
])

In [8]:
def train_predict(full_pipeline, train_X, train_y, test_X, test_y):
    full_pipeline.fit(train_X, train_y)
    pred_y = full_pipeline.predict(test_X)
    print('accuracy_score:', accuracy_score(test_y, pred_y))
    print('classification_report:', classification_report(test_y, pred_y))
    
    print('cross_val_score:')
    scores = cross_val_score(full_pipeline, X, y, cv=5, scoring='accuracy')
    print(scores)
    print('mean of scores:', scores.mean())

In [9]:
from xgboost import XGBClassifier

models = [
   
    ('XGB', XGBClassifier(learning_rate=0.1,
                        n_estimators=100,
                        eval_metric = 'error',
                        max_depth=6,               
                        min_child_weight = 1,      
                        gamma=0.,                  
                        subsample=0.8,
                        use_label_encoder = False,  
                        scale_pos_weight=1,        
                        random_state=27)),
]

for model_name, model in models:
    print('\nModel %s.' % model_name)
    full_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    train_predict(full_pipeline, train_X, train_y, test_X, test_y)
    print('*' * 50)


Model XGB.
accuracy_score: 0.8211382113821138
classification_report:               precision    recall  f1-score   support

           0       0.70      0.58      0.63        33
           1       0.85      0.91      0.88        90

    accuracy                           0.82       123
   macro avg       0.78      0.74      0.76       123
weighted avg       0.81      0.82      0.82       123

cross_val_score:
[0.74796748 0.73170732 0.75609756 0.78861789 0.79508197]
mean of scores: 0.7638944422231106
**************************************************


In [10]:
predicted = full_pipeline.predict(pd.DataFrame(columns=['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area'],data=np.array(['Male', 'Yes', 1 ,'Graduate', 'No', 3 , 0, 1280.0, 36.0, 1.0, 'Rural']).reshape(1,11)))

In [11]:
predicted

array([0])

In [12]:
if(predicted == 1):
    print('loan_approved')
else:
    print("Not_approved")

Not_approved


##IBM DEPLOYMENT

In [13]:
!pip install -U ibm-watson-machine-learning

In [14]:
from ibm_watson_machine_learning import APIClient
import json

##AUTHENTICATE AND SET SPACE

In [15]:
wml_credentials = {
    "apikey":"GtB6VnZkmgTvkfFfwdYa2F9-3S1KGTle_On2TREFW8JZ",
    "url":"https://us-south.ml.cloud.ibm.com"
}


In [16]:
wml_client = APIClient(wml_credentials)

In [17]:
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -----------  ------------------------
ID                                    NAME         CREATED
18374852-e037-4e50-9548-7d917d6d6dc7  loan-deploy  2022-11-19T09:30:03.224Z
------------------------------------  -----------  ------------------------


In [18]:
SPACE_ID= "18374852-e037-4e50-9548-7d917d6d6dc7"

In [19]:
wml_client.set.default_space(SPACE_ID)

'SUCCESS'

In [20]:
wml_client.software_specifications.list(100)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

Save and deploy model

In [21]:
import sklearn
sklearn.__version__

'1.0.2'

In [22]:
MODEL_NAME = 'loan'
DEPLOYMENT_NAME = 'loan-deploy'
DEMO_MODEL = model

In [23]:
# Set Python Version
software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

In [24]:
# Setup model meta
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME, 
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0', 
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid 
}

In [25]:
#Save model

model_details = wml_client.repository.store_model(
    model=DEMO_MODEL, 
    meta_props=model_props, 
    training_data=train_X, 
    training_target=train_y)

In [26]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'label_column': 'l1',
  'schemas': {'input': [{'fields': [{'name': 'Gender', 'type': 'object'},
      {'name': 'Married', 'type': 'object'},
      {'name': 'Dependents', 'type': 'object'},
      {'name': 'Education', 'type': 'object'},
      {'name': 'Self_Employed', 'type': 'object'},
      {'name': 'ApplicantIncome', 'type': 'int64'},
      {'name': 'CoapplicantIncome', 'type': 'float64'},
      {'name': 'LoanAmount', 'type': 'float64'},
      {'name': 'Loan_Amount_Term', 'type': 'float64'},
      {'name': 'Credit_History', 'type': 'float64'},
      {'name': 'Property_Area', 'type': 'object'}],
     'id': '1',
     'type': 'struct'}],
   'output': []},
  'software_spec': {'id': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
   'name': 'runtime-22.1-py3.9'},
  'type': 'scikit-learn_1.0'},
 'metadata': {'created_at': '2022-11-19T09:33:18.172Z',
  'id': '3f8059fb-9d37-45ee-9a83-0d95d0367ecb',
  'modified_at': '2022-11-19T09:33:20.497Z',
  'na

In [27]:
model_id = wml_client.repository.get_model_id(model_details)
model_id

'3f8059fb-9d37-45ee-9a83-0d95d0367ecb'

In [28]:
# Set meta
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME, 
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [29]:

# Deploy
deployment = wml_client.deployments.create(
    artifact_uid=model_id, 
    meta_props=deployment_props 
)




#######################################################################################

Synchronous deployment creation for uid: '3f8059fb-9d37-45ee-9a83-0d95d0367ecb' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='754bf3d0-e2e8-4b7b-ba20-9070fbbf8471'
------------------------------------------------------------------------------------------------


